## Convert the R1CS into a QAP


In [11]:
import numpy as np
import random

# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

print("witness {}".format(w))

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"

witness [         1 1500761992        959        543    2759043 1498160349]


### Calculating Matrix Ua

In [19]:
from scipy.interpolate import lagrange

# A = np.array([[0,0,3,0,0,0],
#                [0,0,0,0,1,0],
#                [0,0,1,0,0,0]])

x = np.array([1,2,3])
y_l3 = np.array([3,0,1])
y_l5 = np.array([0,1,0])

poly_l3 = lagrange(x, y_l3)
poly_l5 = lagrange(x, y_l5)


print(poly_l3)
print(poly_l5)

U = np.array([[0,0,2,0,-1,0],
               [0,0,-9,0,4,0],
               [0,0,10,0,3,0]])

Ua = np.matmul(U, w)

print("Ua {}".format(Ua))




   2
2 x - 9 x + 10
    2
-1 x + 4 x - 3
Ua [-1439361  5756751  4329171]


### Calculating Matrix Va

In [22]:
# B = np.array([[0,0,1,0,0,0],
#                [0,0,0,1,0,0],
#                [0,0,0,5,0,0]])

x = np.array([1,2,3])
y_r3 = np.array([1,0,0])
y_r4 = np.array([0,1,5])

poly_r3 = lagrange(x, y_r3)
poly_r4 = lagrange(x, y_r4)


print(poly_r3)
print(poly_r4)

V = np.array([[0,0,0.5,1.5,0,0],
               [0,0,-2.5,-3.5,0,0],
               [0,0,3,2,0,0]])

Va = np.matmul(V, w)

print("Va {}".format(Va))


     2
0.5 x - 2.5 x + 3
     2
1.5 x - 3.5 x + 2
Va [  910.5 -3048.5  2831. ]


### Calculating Matrix Wa

In [25]:
# C = np.array([[0,0,0,0,1,0],
#                [0,0,0,0,0,1],
#                [-3,1,1,2,0,-1]])

x = np.array([1,2,3])
y_o1 = np.array([0,0,-3])
y_o2 = np.array([0,0,1])
y_o3 = np.array([0,0,1])
y_o4 = np.array([0,0,2])
y_o5 = np.array([1,0,0])
y_o6 = np.array([0,1,-1])

poly_o1 = lagrange(x, y_o1)
poly_o2 = lagrange(x, y_o2)
poly_o3 = lagrange(x, y_o3)
poly_o4 = lagrange(x, y_o4)
poly_o5 = lagrange(x, y_o5)
poly_o6 = lagrange(x, y_o6)

print(poly_o1)
print(poly_o2)
print(poly_o3)
print(poly_o4)
print(poly_o5)
print(poly_o6)

W = np.array([[-1.5,0.5,0.5,1,0.5,-1.5],
               [4.5,-1.5,-1.5,-3,-2.5,5.5],
               [-3,1,1,2,3,-4]])
Wa = np.matmul(W, w)

print("Wa {}".format(Wa))

      2
-1.5 x + 4.5 x - 3
     2
0.5 x - 1.5 x + 1
     2
0.5 x - 1.5 x + 1
   2
1 x - 3 x + 2
     2
0.5 x - 2.5 x + 3
      2
-1.5 x + 5.5 x - 4
Wa [-5.40349078e+08  2.16132736e+09 -1.61953754e+09]


### Calculating h(x) t(x)

In [26]:
from numpy import poly1d

a = poly1d(Ua)
b = poly1d(Va)
c = poly1d(Wa)
t = poly1d([1, -1])*poly1d([1, -2])*poly1d([1, -3])

(a * b - c) / t


(poly1d([-1.31053819e+09,  1.76618465e+09]),
 poly1d([ 7.87080086e+09, -2.63527034e+10,  2.44725285e+10]))

In [12]:
from scipy.interpolate import lagrange
import numpy as np
from numpy.polynomial import polynomial as P

np.set_printoptions(precision=2)


def poly_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = lagrange(index, row).coef[::-1]
        res.append(poly)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))


def QAP(s, A, B, C):
    L = np.matmul(s, A)
    R = np.matmul(s, B)
    O = np.matmul(s, C)
    print(f"L: {L}")
    print(f"R: {R}")
    print(f"O: {O}")
    Q = P.polysub(P.polymul(L, R), O)
    print(f"T:\n {Q}")
    assert len(L) == len(R) == len(O)
    return P.polydiv(Q, Z(len(L)))


def Z(n):
    ns = [[-(1 + i), 1] for i, _ in enumerate(range(n))]
    return polymuln(ns)


def main(s, A, B, C):
    x = np.arange(1, len(A) + 1)

    resA = poly_interpolate(x, np.transpose(A))
    resB = poly_interpolate(x, np.transpose(B))
    resC = poly_interpolate(x, np.transpose(C))

    print(f"interporated A: \n {resA}")
    print(f"interporated B: \n {resB}")
    print(f"interporated C: \n {resC}")

    return QAP(s, resA, resB, resC)




print(main(w, A, B, C))

interporated A: 
 [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
interporated B: 
 [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
interporated C: 
 [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
L: [-8267539. 11027541. -2757125.]
R: [ 3963. -4298.  1294.]
O: [-4483600233.0 5981838261.0 -1495478985.0]
T:
 [-28280656824.0 73254189344.0 -67525574074.0 26119761304.0 -3567719750.0]
(array([4713442804.0, -3567719750.0], dtype=object), array([0.0], dtype=object))
